In [2]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json

In [21]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json?$where=offense_description=BURGLARY- AGGRAVATED' # between 2021-01-01T00:00:00 and 2021-06-30T23:59:59'            

params = {
    'offense_description': 'BURGLARY- AGGRAVATED',
    'incident_reported': between '2021-01-01T00:00:00' and '2021-06-30T23:59:59') 
}

In [22]:
response = requests.get(endpoint)#, params=params)

In [23]:
response.json()

{'message': 'Query coordinator error: query.soql.no-such-column; No such column: BURGLARY; position: Map(row -> 1, column -> 617, line -> "SELECT `primary_key`, `incident_number`, `report_type`, `report_type_description`, `incident_status_code`, `incident_status_description`, `investigation_status`, `incident_occurred`, `incident_reported`, `incident_location`, `latitude`, `longitude`, `zip_code`, `rpa`, `zone`, `location_code`, `location_description`, `offense_number`, `offense_nibrs`, `offense_description`, `weapon_primary`, `weapon_description`, `victim_number`, `domestic_related`, `victim_type`, `victim_description`, `victim_gender`, `victim_race`, `victim_ethnicity`, `victim_county_resident`, `mapped_location` WHERE `offense_description` = `BURGLARY` - `AGGRAVATED`\\n                                                                                                                                                                                                                         

with open ('data/census.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']